In [ ]:
import tensorflow as tf
from pydst.rate_limiters import RateLimited 
from pydst.dataproviders import DataProvider
from IPython.display import display, HTML
import datetime
import os
import numpy as np
def show_graph(graph_def, frame_size=(900, 600)):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:{height}px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(height=frame_size[1], data=repr(str(graph_def)), id='graph'+timestamp)
    iframe = """
        <iframe seamless style="width:{width}px;height:{height}px;border:0" srcdoc="{src}"></iframe>
    """.format(width=frame_size[0], height=frame_size[1] + 20, src=code.replace('"', '&quot;'))
    display(HTML(iframe))

In [ ]:
@RateLimited(1)
def eval_tensor(sess, inputs, targets):
    [b,t] = sess.run([inputs, targets])
    print(b)
    

In [ ]:
def bgraph_t1(which_set, rng=None):
    
    graph = tf.Graph()
    outputs = {}
    errors = {}

    timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    #train_writer = tf.summary.FileWriter(os.path.join('tf-log', timestamp, 'train'), graph=graph)
    #valid_writer = tf.summary.FileWriter(os.path.join('tf-log', timestamp, 'valid'), graph=graph)
    
    with graph.as_default():

        with tf.device('/cpu:0'):
            dataprovider = DataProvider(graph=graph, which_set=which_set, batch_size=10, down_sample=1, target_size=6,rng=rng)
            inputs, targets = dataprovider.get_data()

        name = 'layer-1'
        with tf.variable_scope(name):
            outputs[name] = tf.layers.dense(inputs=inputs, units=100, name=name, activation=tf.nn.relu)

        name = 'layer-2'
        with tf.variable_scope(name):
            outputs[name] = tf.layers.dense(inputs=outputs['layer-1'], units=6, name=name, activation=tf.nn.relu)

        name = 'layer-3'
        with tf.variable_scope(name):
            outputs[name] = tf.layers.dense(inputs=outputs['layer-2'], units=6, name=name, activation=tf.nn.relu)

        name = 'layer-4'
        with tf.variable_scope(name):
            outputs[name] = tf.layers.dense(inputs=outputs['layer-3'], units=6, name=name, activation=tf.nn.relu)

        name = 'error-l4'
        with tf.variable_scope(name):
            errors[name] = tf.losses.softmax_cross_entropy(onehot_labels=targets, logits=outputs['layer-4'])

        name = 'error-l2'
        with tf.variable_scope(name):
            errors[name] = tf.losses.softmax_cross_entropy(onehot_labels=targets, logits=outputs['layer-2'])
        train_step = tf.train.AdamOptimizer().minimize(errors['error-l4'])

        tf.summary.scalar('error-l2', errors['error-l2'])
        tf.summary.scalar('error-l4', errors['error-l4'])
        summary_op = tf.summary.merge_all()
        saver = tf.train.Saver()
        
        sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=8))
        
    return sess, graph, summary_op, train_step, dataprovider, saver, inputs, targets


In [ ]:
with tf.variable_scope("model_t1") as scope:
    [train_sess, train_graph, train_summary, train_step, trainData, trainSaver, 
                                                                tinputs, ttargets] = bgraph_t1('train')
    
    scope.reuse_variables()
    rng = np.random.RandomState(15679832)
    [valid_sess, valid_graph, valid_summary, _, validData, validSaver, 
                                                             vinputs, vtargets] = bgraph_t1('valid', rng)

In [ ]:
with train_graph.as_default():
    
    train_sess.run(tf.global_variables_initializer())
    trainData.enable(train_sess)
    
    for num in range(1):
        eval_tensor(train_sess, tinputs, ttargets)
    trainData.disable(train_sess)
    #saver.export_meta_graph('graph_model.meta')
    #saver.save(train_sess, 'my-model')

In [ ]:
with valid_graph.as_default():
    
    valid_sess.run(tf.global_variables_initializer())
    validData.enable(valid_sess)
    
    for num in range(1):
        eval_tensor(valid_sess, vinputs, vtargets)
    validData.disable(valid_sess)
    #saver.export_meta_graph('graph_model.meta')
    #saver.save(valid_sess, 'my-model')

In [ ]:
show_graph(train_graph)
show_graph(valid_graph)